In [1]:
import pandas as pd
import textwrap
import numpy as np

from sentence_transformers import SentenceTransformer

/Users/zzz/Downloads/anaconda3/envs/bancobv_entrevista/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-12 23:48:23.823652: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# carregar modelos
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", trust_remote_code=True)

In [3]:
df = pd.read_csv('../dados/analise_de_sentimentos.csv')
# transferir sentimento pra texto
df['sentimento_tweet'] = np.where(df['sentimento_tweet'] == 0, 'negativo',np.where(df['sentimento_tweet'] == 1,'neutro','positivo'))

In [4]:
# chuck
text = ''
for index,row in df.iterrows():
   text+=f'''{{
   'id':'{row['id']}',
   'text of tweet':'{row['texto_tweet']}',
   'idioma of tweet':'{row['idioma_tweet']}',
   'sentiment of tweet':'{row['sentimento_tweet']}'
   }},
   
   '''
text = text[:-5]
# text

paragraphs = text.split('\n   \n   ') 
# paragraphs

In [5]:
for i, p in enumerate(paragraphs):
    wrapped_text = textwrap.fill(p, width=100)
    print("-----------------------------------------------------------------")
    print(wrapped_text)
    print("-----------------------------------------------------------------")

-----------------------------------------------------------------
{    'id':'595e60b48fcd022a715f7b7b',    'text of tweet':'this @airfrance b777-300er has the oldest
ifes i've ever seen. it belongs in a museum. the terrible smell isn't helping either.',    'idioma
of tweet':'en',    'sentiment of tweet':'negativo'    },
-----------------------------------------------------------------
-----------------------------------------------------------------
{    'id':'595e60de8fcd022a715f7b7d',    'text of tweet':'???? will miss my connection @airfrance
https://t.co/2olmtwcxyk',    'idioma of tweet':'en',    'sentiment of tweet':'negativo'    },
-----------------------------------------------------------------
-----------------------------------------------------------------
{    'id':'595e61448fcd022a715f7b7f',    'text of tweet':'@airfrance lost luggage in overhead cabin,
email no response, phone no one answers. pls help.',    'idioma of tweet':'en',    'sentiment of
tweet':'negativo'    },


In [6]:
# rag
docs_embed = model.encode(paragraphs, normalize_embeddings=True)
print(docs_embed.shape)  # embedding output

(1096, 768)


In [7]:
# save text
with open('paragraphs.txt', 'w') as output:
    output.write(str(paragraphs))

In [8]:
# save vetores
np.save('embeddings_tweet_hist.npy', docs_embed)